### Linear Regression Model for predicting Bike Availability: 


- Here we will be implementing a linear regression model in order to predict the number of bikes available and the number of bike stands available at a give bike stand. 
- Linear regression is a statistical method for modeling relationships between a dependent variable with a given set of independent variables.
- In our model the dependent variable will be number of bikes/bike stands and the independent variables will be time of day, day of the week, area, and weather.  

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine

import pickle

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


from sklearn.ensemble import RandomForestRegressor





### Connect to database:

In [2]:
URL = "dublin-bikesdb.cmd8vuwgew1e.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbikes"
USER = "admin"
PASSWORD = "Dbikes123"


### Weather Data 

In [3]:
def weather():
    engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)
    sql_query_weather= """
    SELECT weather.id, weather.description1, weather.temperature, weather.humidity, weather.windspeed, weather.sunset, weather.TIME FROM weather;
    """
    df_weather = pd.read_sql_query(sql_query_weather, engine)

    return df_weather


df_weather = weather()

2022-04-02 13:49:10,791 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-04-02 13:49:10,791 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:10,921 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-04-02 13:49:10,926 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ()
2022-04-02 13:49:11,494 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-04-02 13:49:11,494 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:12,310 INFO sqlalchemy.engine.Engine 
    SELECT weather.id, weather.description1, weather.temperature, weather.humidity, weather.windspeed, weather.sunset, weather.TIME FROM weather;
    
2022-04-02 13:49:12,320 INFO sqlalchemy.engine.Engine [raw sql] ()


In [4]:
df_weather

,id,description1,temperature,humidity,windspeed,sunset,TIME
0,803,broken clouds,279.49,78,8.23,2022-03-30 18:55:15,2022-03-30 15:29:16
1,803,broken clouds,279.49,78,8.23,2022-03-30 18:55:15,2022-03-30 15:34:17
2,803,broken clouds,279.61,78,8.23,2022-03-30 18:55:15,2022-03-30 15:39:17
3,803,broken clouds,279.61,78,8.23,2022-03-30 18:55:15,2022-03-30 15:44:17
4,803,broken clouds,279.45,78,7.20,2022-03-30 18:55:15,2022-03-30 15:49:18
...,...,...,...,...,...,...,...
827,802,scattered clouds,281.95,66,5.14,2022-04-02 19:00:38,2022-04-02 12:27:37
828,802,scattered clouds,281.95,66,5.14,2022-04-02 19:00:38,2022-04-02 12:32:37
829,802,scattered clouds,282.11,67,5.14,2022-04-02 19:00:39,2022-04-02 12:37:38
830,802,scattered clouds,282.14,68,5.14,2022-04-02 19:00:39,2022-04-02 12:42:38


In [5]:
df_weather['just_date'] = df_weather['sunset'].dt.date
df_weather

,id,description1,temperature,humidity,windspeed,sunset,TIME,just_date
0,803,broken clouds,279.49,78,8.23,2022-03-30 18:55:15,2022-03-30 15:29:16,2022-03-30
1,803,broken clouds,279.49,78,8.23,2022-03-30 18:55:15,2022-03-30 15:34:17,2022-03-30
2,803,broken clouds,279.61,78,8.23,2022-03-30 18:55:15,2022-03-30 15:39:17,2022-03-30
3,803,broken clouds,279.61,78,8.23,2022-03-30 18:55:15,2022-03-30 15:44:17,2022-03-30
4,803,broken clouds,279.45,78,7.20,2022-03-30 18:55:15,2022-03-30 15:49:18,2022-03-30
...,...,...,...,...,...,...,...,...
827,802,scattered clouds,281.95,66,5.14,2022-04-02 19:00:38,2022-04-02 12:27:37,2022-04-02
828,802,scattered clouds,281.95,66,5.14,2022-04-02 19:00:38,2022-04-02 12:32:37,2022-04-02
829,802,scattered clouds,282.11,67,5.14,2022-04-02 19:00:39,2022-04-02 12:37:38,2022-04-02
830,802,scattered clouds,282.14,68,5.14,2022-04-02 19:00:39,2022-04-02 12:42:38,2022-04-02


In [6]:
df_weather.dtypes


id                       int64
description1            object
temperature            float64
humidity                 int64
windspeed              float64
sunset          datetime64[ns]
TIME            datetime64[ns]
just_date               object
dtype: object

### Availablity Data

In [7]:
def availability():
    engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)
    df_avail = pd.read_sql_table("availability", engine)
    return df_avail

In [8]:
df_avail = availability()

2022-04-02 13:49:15,251 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-04-02 13:49:15,251 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:15,478 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-04-02 13:49:15,494 INFO sqlalchemy.engine.Engine [generated in 0.00211s] ()
2022-04-02 13:49:15,915 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-04-02 13:49:15,915 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:17,122 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `dbikes`
2022-04-02 13:49:17,122 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:17,436 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `dbikes`
2022-04-02 13:49:17,436 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:17,767 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `availability`
2022-04-02 13:49:17,767 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 13:49:18,178 INFO sqlalchemy.engine.Engine SELECT availabili

In [9]:
df_avail

,number,available_bike_stands,available_bikes,last_update
0,42,16,14,2022-02-23 19:50:20
1,30,0,20,2022-02-23 19:41:25
2,54,11,22,2022-02-23 19:48:38
3,108,16,19,2022-02-23 19:51:13
4,56,2,38,2022-02-23 19:45:20
...,...,...,...,...
942054,39,6,14,2022-04-02 12:41:13
942055,83,14,26,2022-04-02 12:47:38
942056,92,30,10,2022-04-02 12:47:56
942057,21,19,11,2022-04-02 12:42:23


In [10]:
df_avail['day'] = df_avail['last_update'].dt.day_of_week
df_avail['hour'] = df_avail['last_update'].dt.hour
df_avail['just_date'] = df_avail['last_update'].dt.date




In [11]:
df_avail

,number,available_bike_stands,available_bikes,last_update,day,hour,just_date
0,42,16,14,2022-02-23 19:50:20,2,19,2022-02-23
1,30,0,20,2022-02-23 19:41:25,2,19,2022-02-23
2,54,11,22,2022-02-23 19:48:38,2,19,2022-02-23
3,108,16,19,2022-02-23 19:51:13,2,19,2022-02-23
4,56,2,38,2022-02-23 19:45:20,2,19,2022-02-23
...,...,...,...,...,...,...,...
942054,39,6,14,2022-04-02 12:41:13,5,12,2022-04-02
942055,83,14,26,2022-04-02 12:47:38,5,12,2022-04-02
942056,92,30,10,2022-04-02 12:47:56,5,12,2022-04-02
942057,21,19,11,2022-04-02 12:42:23,5,12,2022-04-02


In [12]:
df_avail.corr()

,number,available_bike_stands,available_bikes,day,hour
number,1.000000,0.080299,0.339412,0.000102,-0.000514
available_bike_stands,0.080299,1.000000,-0.675170,0.007901,-0.014565
available_bikes,0.339412,-0.675170,1.000000,-0.008833,0.013541
day,0.000102,0.007901,-0.008833,1.000000,-0.039794
hour,-0.000514,-0.014565,0.013541,-0.039794,1.000000


In [13]:
df_avail.groupby(['day']).mean()

,number,available_bike_stands,available_bikes,hour
day,,,,
0,60.274707,12.590078,19.282253,11.582122
1,60.371049,12.570528,19.421478,12.347305
2,60.325617,12.704824,18.971550,11.481926
3,60.323107,12.590797,18.825266,12.605026
4,60.331106,12.634722,19.086406,11.506133
5,60.328908,12.811904,19.036325,10.847453
6,60.325019,12.809722,18.988247,11.508759


In [14]:
df_avail.dtypes


number                            int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
day                               int64
hour                              int64
just_date                        object
dtype: object

In [15]:
df_avail["number"] = df_avail["number"].astype('category')  


In [16]:
df_avail.shape

(942059, 7)

In [17]:
df_avail.describe().T

,count,mean,std,min,25%,50%,75%,max
available_bike_stands,942059.0,12.673264,9.072681,0.0,5.0,12.0,19.0,40.0
available_bikes,942059.0,19.068313,10.619497,0.0,11.0,19.0,27.0,40.0
day,942059.0,3.204208,1.818348,0.0,2.0,3.0,5.0,6.0
hour,942059.0,11.683217,6.906362,0.0,6.0,12.0,18.0,23.0


In [18]:
df_avail["number"].describe().T

count     942059
unique       110
top           61
freq        8566
Name: number, dtype: int64

#### Combining the two data frames 

In [19]:
# df_combine= df_avail.merge(df_weather,join='inner', on=['just_date'])

df_combine = pd.merge(df_avail, df_weather, on='just_date', how="outer")

# df_weather = df_weather.just_date.map(df_avail.set_index('just_date')
                                      
# pd.merge_asof(df_weather, df_avail, left_on='just_date', right_on='just_date')


# df_combine= df_avail.merge(df_weather,on=['just_date'])

# inner_merged = pd.concat([df_avail, df_weather],on=['just_date'])

# v = df1.merge(df2[['Date', 'exp']])\
#        .groupby(df1.columns.tolist())\
#        .exp\
#        .apply(pd.Series.tolist)


In [20]:
df_weather

,id,description1,temperature,humidity,windspeed,sunset,TIME,just_date
0,803,broken clouds,279.49,78,8.23,2022-03-30 18:55:15,2022-03-30 15:29:16,2022-03-30
1,803,broken clouds,279.49,78,8.23,2022-03-30 18:55:15,2022-03-30 15:34:17,2022-03-30
2,803,broken clouds,279.61,78,8.23,2022-03-30 18:55:15,2022-03-30 15:39:17,2022-03-30
3,803,broken clouds,279.61,78,8.23,2022-03-30 18:55:15,2022-03-30 15:44:17,2022-03-30
4,803,broken clouds,279.45,78,7.20,2022-03-30 18:55:15,2022-03-30 15:49:18,2022-03-30
...,...,...,...,...,...,...,...,...
827,802,scattered clouds,281.95,66,5.14,2022-04-02 19:00:38,2022-04-02 12:27:37,2022-04-02
828,802,scattered clouds,281.95,66,5.14,2022-04-02 19:00:38,2022-04-02 12:32:37,2022-04-02
829,802,scattered clouds,282.11,67,5.14,2022-04-02 19:00:39,2022-04-02 12:37:38,2022-04-02
830,802,scattered clouds,282.14,68,5.14,2022-04-02 19:00:39,2022-04-02 12:42:38,2022-04-02


In [21]:
df_combine
# inner_merged

,number,available_bike_stands,available_bikes,last_update,day,hour,just_date,id,description1,temperature,humidity,windspeed,sunset,TIME
0,42,16,14,2022-02-23 19:50:20,2,19,2022-02-23,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,30,0,20,2022-02-23 19:41:25,2,19,2022-02-23,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,54,11,22,2022-02-23 19:48:38,2,19,2022-02-23,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,108,16,19,2022-02-23 19:51:13,2,19,2022-02-23,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,56,2,38,2022-02-23 19:45:20,2,19,2022-02-23,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22102418,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,281.95,66.0,5.14,2022-04-02 19:00:38,2022-04-02 12:27:37
22102419,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,281.95,66.0,5.14,2022-04-02 19:00:38,2022-04-02 12:32:37
22102420,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,282.11,67.0,5.14,2022-04-02 19:00:39,2022-04-02 12:37:38
22102421,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,282.14,68.0,5.14,2022-04-02 19:00:39,2022-04-02 12:42:38


In [22]:
df_combine.tail(40)

,number,available_bike_stands,available_bikes,last_update,day,hour,just_date,id,description1,temperature,humidity,windspeed,sunset,TIME
22102383,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.04,89.0,5.66,2022-04-02 19:00:39,2022-04-02 09:32:31
22102384,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.23,88.0,5.66,2022-04-02 19:00:39,2022-04-02 09:37:31
22102385,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.43,88.0,5.66,2022-04-02 19:00:39,2022-04-02 09:42:31
22102386,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.74,85.0,4.63,2022-04-02 19:00:39,2022-04-02 09:47:31
22102387,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.74,85.0,4.63,2022-04-02 19:00:39,2022-04-02 09:52:32
22102388,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.80,85.0,4.63,2022-04-02 19:00:39,2022-04-02 09:57:32
22102389,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,279.80,85.0,4.63,2022-04-02 19:00:39,2022-04-02 10:02:32
22102390,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,280.09,82.0,4.63,2022-04-02 19:00:39,2022-04-02 10:07:32
22102391,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,280.09,82.0,4.63,2022-04-02 19:00:39,2022-04-02 10:12:33
22102392,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02,802.0,scattered clouds,280.11,82.0,4.63,2022-04-02 19:00:38,2022-04-02 10:17:33


### Looking at correlations

In [23]:
df_combine.corr()

,available_bike_stands,available_bikes,day,hour,id,temperature,humidity,windspeed
available_bike_stands,1.000000,-0.698968,0.007076,-0.011283,-0.001478,0.000856,0.006116,-0.002845
available_bikes,-0.698968,1.000000,-0.007248,0.008871,0.001873,-0.000250,-0.005763,0.003383
day,0.007076,-0.007248,1.000000,-0.223689,-0.266949,-0.064480,0.581764,-0.501357
hour,-0.011283,0.008871,-0.223689,1.000000,0.020330,-0.022416,-0.108038,0.115840
id,-0.001478,0.001873,-0.266949,0.020330,1.000000,-0.147757,-0.352129,0.064821
temperature,0.000856,-0.000250,-0.064480,-0.022416,-0.147757,1.000000,-0.623433,0.056020
humidity,0.006116,-0.005763,0.581764,-0.108038,-0.352129,-0.623433,1.000000,-0.246392
windspeed,-0.002845,0.003383,-0.501357,0.115840,0.064821,0.056020,-0.246392,1.000000


In [24]:
df_avail.head(5)

,number,available_bike_stands,available_bikes,last_update,day,hour,just_date
0,42,16,14,2022-02-23 19:50:20,2,19,2022-02-23
1,30,0,20,2022-02-23 19:41:25,2,19,2022-02-23
2,54,11,22,2022-02-23 19:48:38,2,19,2022-02-23
3,108,16,19,2022-02-23 19:51:13,2,19,2022-02-23
4,56,2,38,2022-02-23 19:45:20,2,19,2022-02-23


In [25]:
df_avail.tail(5)

,number,available_bike_stands,available_bikes,last_update,day,hour,just_date
942054,39,6,14,2022-04-02 12:41:13,5,12,2022-04-02
942055,83,14,26,2022-04-02 12:47:38,5,12,2022-04-02
942056,92,30,10,2022-04-02 12:47:56,5,12,2022-04-02
942057,21,19,11,2022-04-02 12:42:23,5,12,2022-04-02
942058,88,6,24,2022-04-02 12:38:58,5,12,2022-04-02


In [26]:
df_avail.shape

(942059, 7)

### Training Model

In [27]:
train_feature = ["day","hour","number"]
target_feature1 = ['available_bikes']
target_feature2 = ['available_bike_stands']

train = df_avail[train_feature]
target = df_avail[target_feature1]
Xtrain,Xtest,Ytrain,Ytest = train_test_split(train,target,test_size=0.3)
Xtrain

,day,hour,number
747093,5,19,102
830417,1,10,3
404598,1,22,84
506827,5,4,84
337163,6,19,106
...,...,...,...
262676,4,10,21
723494,5,1,57
67782,4,9,45
829722,1,10,25


In [28]:
LR = LinearRegression()
LR.fit(Xtrain,Ytrain)


C:\Users\michael\anaconda3\envs\comp30830\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [29]:
LR.score(Xtest,Ytest)

C:\Users\michael\anaconda3\envs\comp30830\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.1162864793291295

In [30]:
y_prediction =  LR.predict(Xtest)
y_prediction

C:\Users\michael\anaconda3\envs\comp30830\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([[20.60096719],
       [25.12040127],
       [24.03178093],
       ...,
       [16.53876862],
       [15.52293324],
       [16.77321482]])

### Example of Prediction

- on Monday, 2pm, station: 100

In [31]:
LR.predict([[1, 14, 100]])

array([[23.44278956]])

### Metrics for model evaluation in linear regression:  

#### R Square/Adjusted R Square

In [32]:
score=r2_score(Ytest,y_prediction)
print('r2 score is ',score)

r2 score is  0.1162864793291295


#### Mean Square Error(MSE)

In [33]:
print('MSE is ',mean_squared_error(Ytest,y_prediction))

MSE is  99.5394151771574


#### Mean Absolute Error

In [34]:
print('Mean Absolute Error is ',mean_absolute_error(Ytest,y_prediction))

Mean Absolute Error is  8.131979424373055


## Using a Random Forest Model

In [35]:
random_forest = RandomForestRegressor(random_state=0)
random_forest = random_forest.fit(Xtrain,Ytrain.available_bikes)
score = random_forest.score(Xtest,Ytest.available_bikes)
score

C:\Users\michael\anaconda3\envs\comp30830\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\michael\anaconda3\envs\comp30830\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.6634240734027689

In [36]:
y_prediction_RF =  random_forest.predict(Xtest)
y_prediction_RF

C:\Users\michael\anaconda3\envs\comp30830\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([21.72990665, 14.15085695, 23.90604883, ..., 14.87573145,
        3.50124368, 14.33583988])

### Metrics for model evaluation in random forrest:  

#### R squared

In [37]:
score=r2_score(Ytest,y_prediction_RF)
print('r2 score is ',score)

r2 score is  0.6634240734027689


#### Mean squared error

In [38]:
print('MSE is ',mean_squared_error(Ytest,y_prediction_RF))

MSE is  37.911121774808635


#### Mean absolute error

In [39]:
print('Mean Absolute Error is ',mean_absolute_error(Ytest,y_prediction_RF))

Mean Absolute Error is  4.724959025380236


### Example of prediction

In [40]:
random_forest.predict([[1, 14, 100]])

array([23.35601927])

### Saving model to disk with Pickle:

In [41]:
pickle.dump(random_forest, open('model.pkl', 'wb'))

In [42]:
#testing

In [43]:
model= pickle.load(open('model.pkl', 'rb'))